In [1]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Model
import seaborn as sns
import io
from keras import optimizers
from keras.callbacks import  ModelCheckpoint
import os

#Principe de la détection d'anomalies par auto-encodeurs:

#On essaye de reconstruire toutes les séquences du signal, de taille fixée, grâce à note auto-encodeur (voir l'image des 1 pour mieux comprendre)
#Si le singal reconstruit par l'auto-encodeur est trop différent du signal de départ alors on dit qu'il y a anomalie

#Autre méthode à creuser pour la détection d'anomalie:

#transformer le signal en image, appliquer une transformée de fourier sur la série temporelle et utiliser un réseau Unet

2022-10-02 19:13:59.760098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-02 19:13:59.760122: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#On implémente sur les données complètes de gamma

df = pd.read_csv('/home/lucien/Documents/projet-data-science-pollution-master/donnees_txt_par_mois/2015_gamma_concat.txt')

#on remarque qu'il y a un très gros pics vers 1633
# for val in df['gamma']:
#     #si la valeur dépasse 1000 afficher le numéro de la ligne
#     if val > 1000:
#         print(df.index[df['gamma'] == val].tolist())

#sns.lineplot(x=df['Date'], y=df['gamma'])
#plt.show()

#Si on avait pas déjà séparer train et test lors de la création des csv on le fait avec pandas, mais c'est plus long

train, test = df.loc[df['Date'] <= 130000], df.loc[df['Date'] > 130000] #on coupe à 130 000 pour avoir un des gros pics que l'on remarque sur le graphique


#Convert pandas dataframe to numpy array
#dataset = dataframe.values
#dataset = dataset.astype('float32') #COnvert values to float

#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized

# normalize the dataset
#scaler = MinMaxScaler() #Also try QuantileTransformer
scaler = StandardScaler()
scaler = scaler.fit(train[['gamma']]) #Compute the mean and std to be used for later scaling.

#on centre réduit nos valeurs

train['gamma'] = scaler.transform(train[['gamma']])
test['gamma'] = scaler.transform(test[['gamma']])


#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 

seq_size = 450  # Number of time steps to look back (on choisit 300 en s'appuyant sur notre travail avec la matrix profile)


#Larger sequences (look further back) may improve forecasting.


def to_sequences(x, y, seq_size=1):
    x_values = []
    y_values = []

    for i in range(len(x)-seq_size):
        #print(i)
        x_values.append(x.iloc[i:(i+seq_size)].values)
        y_values.append(y.iloc[i+seq_size])
        
    return np.array(x_values), np.array(y_values)

trainX, trainY = to_sequences(train[['gamma']], train['gamma'], seq_size) #on découpe en séquence de 300
testX, testY = to_sequences(test[['gamma']], test['gamma'], seq_size)


#ici pas auto-encodeur amis auto-régressif car on prédit pas un timestamp n à partir du n-ième mais on prédit n+1

#REGARDER wave net

/tmp/ipykernel_200506/61914224.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['gamma'] = scaler.transform(train[['gamma']])
/tmp/ipykernel_200506/61914224.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['gamma'] = scaler.transform(test[['gamma']])


In [3]:
# Define Autoencoder model

#first model
#Input shape would be seq_size, 1 - 1 beacuse we have 1 feature. 
# seq_size = trainX.shape[1]

# model = Sequential()
# model.add(LSTM(256, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
# model.add(LSTM(128, activation='relu', return_sequences=False))
# model.add(RepeatVector(trainX.shape[1]))
# model.add(LSTM(128, activation='relu', return_sequences=True))
# model.add(LSTM(256, activation='relu', return_sequences=True))
# model.add(TimeDistributed(Dense(trainX.shape[2])))

# model.compile(optimizer='adam', loss='mse')
# model.summary()

#second model
#Try another model with one LSTM layer

model = Sequential()
model.add(LSTM(256, input_shape=(trainX.shape[1], trainX.shape[2]))) #128 car on a 128 neurones dans la couche cachée (on va augmenter car 128 peut-être trop faible, c'est peut-être la raison de notre under-fitting)
model.add(Dropout(rate=0.2)) #rate?

model.add(RepeatVector(trainX.shape[1]))

model.add(LSTM(256, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(trainX.shape[2])))

#ici on devriat normalement mettre une couche avec une foction d'activation, mais là on met une fonction linéaire par défaut

#optim = tf.keras.optimizers.SGD(learning_rate = 1)
model.compile(optimizer='adam', loss='mse') #on rajoute un optimizer et une loss
model.summary()

2022-10-02 19:14:19.650426: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-02 19:14:19.650449: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-C8Q1Q308): /proc/driver/nvidia/version does not exist
2022-10-02 19:14:19.650788: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256)               264192    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 repeat_vector (RepeatVector  (None, 450, 256)         0         
 )                                                               
                                                                 
 lstm_1 (LSTM)               (None, 450, 256)          525312    
                                                                 
 dropout_1 (Dropout)         (None, 450, 256)          0         
                                                                 
 time_distributed (TimeDistr  (None, 450, 1)           257       
 ibuted)                                                

In [5]:
#on sauvegarde les poids pour les charger plus tard

checkpoint_path = "/home/lucien/Documents/projet-data-science-pollution-master/weights/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


In [6]:
#on lance le training

# fit model
history = model.fit(trainX, trainY, epochs=20, batch_size=32, validation_split=0.1, shuffle=False, callbacks=[cp_callback]) #nombre faible d'epoch pour l'instant, on veut juste tester


Epoch 1/20


2022-10-02 19:15:04.807371: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 209871000 exceeds 10% of free system memory.


  17/3644 [..............................] - ETA: 1:27:18 - loss: 0.1753

In [ ]:
#On trace le résultat de notre train

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()


In [ ]:
#On évalue notre modèle sur l'ensemble test en utilisant les metrics de base

model.evaluate(testX, testY) #on évalue le training 

In [ ]:
###########################
#Passons maintenant à la détection des anomalies sur notre propre signal, pour cela il est capital de définir un seuil de détection d'anomalie.
#Pour commencer un fixe un treshold simple: 90% de la valeur de l'erreur maxime (au delà on considèrera qu'on aura une anomalie)

trainPredict = model.predict(trainX)
trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
plt.hist(trainMAE, bins=30) #donne  la moyenne des erreurs absolues calculée sur chaque channel de taille 300 qu'on a voulu prédire
max_trainMAE = 0.5  #or Define 90% value of max as threshold. On l'a choisi à partir du graphique obtenu ci-dessus. (on prend la dernière valeur de l'abcisse de l'histogramme bleu qu'on vient de tracer)

testPredict = model.predict(testX)
testMAE = np.mean(np.abs(testPredict - testX), axis=1)
plt.hist(testMAE, bins=30) #trace un histogramme rouge

#Capture all details in a DataFrame for easy plotting
anomaly_df = pd.DataFrame(test[seq_size:])
anomaly_df['testMAE'] = testMAE
anomaly_df['max_trainMAE'] = max_trainMAE
anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE'] #si cette inégalité est vérifié alors il y a anomalie car l'erreur moyenne est supérieure au treshold
anomaly_df['gamma'] = test[seq_size:]['gamma']

print(anomaly_df)

anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]


In [ ]:
#On trace l'évolution de l'erreur moyenne en fonction du temps
sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['max_trainMAE'], color='g')#trace la ligne égale au treshold
sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['testMAE']) 

In [ ]:
#0n trace notre signal test en bleu et les anomalies sur notre échantillon test

sns.lineplot(x=anomaly_df['Date'], y=anomaly_df['gamma'], color='b')#trace les valeurs de gamma pour l'échantillon test
sns.scatterplot(x=anomalies['Date'], y=anomalies['gamma'], color='r') #trace les points

In [ ]:
print(anomalies['Date'])